<a href="https://colab.research.google.com/github/MadhuShree-A/Fake-Job-Posting-Prediction/blob/main/Fake_Job_Posting_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EDA before Preprocessing

In [ ]:
# eda

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# 1. LOAD DATA
df = pd.read_csv("fake_job_postings.csv", encoding="utf-8", on_bad_lines="skip")

# 2. TEXT STATS
text_cols = ["title", "company_profile", "description", "requirements", "benefits"]

for col in text_cols:
    df[f"{col}_wordcount"] = df[col].fillna("").apply(lambda x: len(str(x).split()))
    df[f"{col}_charcount"] = df[col].fillna("").apply(lambda x: len(str(x)))

# Hist: Description length by fraud
# HISTOGRAMS: Word Count Distribution by Fraud Status (All Text Fields)
plt.figure(figsize=(15,12))
for i, col in enumerate(text_cols, 1):
    plt.subplot(3, 2, i)
    sns.histplot(data=df, x=f"{col}_wordcount", hue="fraudulent", bins=50, alpha=0.6)
    plt.title(f"Distribution of {col.capitalize()} Word Count by Fraud Status")
    plt.xlabel("Word Count")
    plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


# Boxplot: All text fields
plt.figure(figsize=(10,6))
long_form = df.melt(id_vars="fraudulent", value_vars=[f"{col}_wordcount" for col in text_cols],
                    var_name="TextField", value_name="WordCount")
sns.boxplot(data=long_form, x="TextField", y="WordCount", hue="fraudulent")
plt.title("Word Count Distribution by Fraud Status (All Text Fields)")
plt.xticks(rotation=45)
plt.ylim(0, 1000)
plt.show()


# 3. WORD CLOUDS & TOP WORDS
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z\s]", "", text)
    return " ".join([w for w in text.split() if w not in stop_words])

fraud_text = " ".join(df[df["fraudulent"]==1]["description"].dropna().apply(clean_text))
real_text  = " ".join(df[df["fraudulent"]==0]["description"].dropna().apply(clean_text))

fraud_wc = WordCloud(width=800, height=400, background_color="black").generate(fraud_text)
real_wc  = WordCloud(width=800, height=400, background_color="white").generate(real_text)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.imshow(fraud_wc, interpolation="bilinear")
plt.axis("off"); plt.title("Fraudulent Postings WordCloud")
plt.subplot(1,2,2)
plt.imshow(real_wc, interpolation="bilinear")
plt.axis("off"); plt.title("Genuine Postings WordCloud")
plt.show()

# Top 20 frequent words (Fraud vs Genuine)
def top_words(text, n=20):
    words = text.split()
    return Counter(words).most_common(n)

print("\nTop 20 Fraudulent Words:\n", top_words(fraud_text, 20))
print("\nTop 20 Genuine Words:\n", top_words(real_text, 20))

# 4. CORRELATION ANALYSIS
num_cols = ["telecommuting", "has_company_logo", "has_questions", "fraudulent"]
corr = df[num_cols].corr()

plt.figure(figsize=(6,4))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap (Numeric Features)")
plt.show()

# 5. CROSS-FEATURE INSIGHTS
# Fraud rate by employment type
plt.figure(figsize=(8,5))
sns.barplot(data=df, x="employment_type", y="fraudulent", estimator=np.mean)
plt.title("Fraud Rate by Employment Type")
plt.ylabel("Fraudulent Rate")
plt.xticks(rotation=45)
plt.show()

# Fraud rate by required experience
plt.figure(figsize=(8,5))
sns.barplot(data=df, x="required_experience", y="fraudulent", estimator=np.mean)
plt.title("Fraud Rate by Required Experience")
plt.ylabel("Fraudulent Rate")
plt.xticks(rotation=45)
plt.show()

# Fraud rate by required education
plt.figure(figsize=(8,5))
sns.barplot(data=df, x="required_education", y="fraudulent", estimator=np.mean)
plt.title("Fraud Rate by Required Education")
plt.ylabel("Fraudulent Rate")
plt.xticks(rotation=45)
plt.show()

# Fraud rate by industry
top_industries = df["industry"].value_counts().head(10).index
plt.figure(figsize=(10,6))
sns.barplot(data=df[df["industry"].isin(top_industries)],
            x="industry", y="fraudulent", estimator=np.mean)
plt.title("Fraud Rate by Industry (Top 10)")
plt.ylabel("Fraudulent Rate")
plt.xticks(rotation=75)
plt.show()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define categorical columns you want to analyze
categorical_cols = ['employment_type', 'has_company_logo', 'telecommuting', 'required_experience']

# Loop through all pairwise combinations
for i, col1 in enumerate(categorical_cols):
    for col2 in categorical_cols[i+1:]:
        # Create pivot table
        pivot = pd.pivot_table(df, values="fraudulent",
                               index=col1,
                               columns=col2,
                               aggfunc=np.mean)

        # Print pivot table
        print(f"\nFraud Rate by {col1} & {col2}:\n")
        print(pivot)

        # Plot heatmap
        plt.figure(figsize=(8,5))
        sns.heatmap(pivot, annot=True, fmt=".2f", cmap="YlOrRd")
        plt.title(f"Fraud Rate: {col1} vs {col2}")
        plt.ylabel(col1)
        plt.xlabel(col2)
        plt.show()

# 7. BINARY COLUMN DISTRIBUTION (Fraud vs Genuine)
binary_cols = ["telecommuting", "has_company_logo", "has_questions"]

for col in binary_cols:
    plt.figure(figsize=(6,4))
    sns.countplot(data=df, x=col, hue="fraudulent")
    plt.title(f"Distribution of {col} by Fraudulent Status")
    plt.xlabel(f"{col} (0=No, 1=Yes)")
    plt.ylabel("Count")
    plt.legend(title="Fraudulent", labels=["Genuine", "Fraud"])
    plt.show()



# Fraud Summary Table
cat_cols = ["employment_type", "required_experience", "required_education", "industry"]
for col in cat_cols:
    summary = df.groupby(col)["fraudulent"].mean().sort_values(ascending=False)
    print(f"\nFraud Rate by {col}:")
    print(summary.head(10))

Preprocessing

In [ ]:

#  PREPROCESSING
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from scipy import sparse
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import pickle

# Download NLTK resources
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

#  Load dataset

print("\nLoading dataset...")

df = pd.read_csv('fake_job_postings.csv')
print(f"✓ Dataset loaded successfully: {df.shape}")
print(f"Fraudulent jobs: {df['fraudulent'].sum()} ({df['fraudulent'].mean()*100:.2f}%)")
print(f"⚠  HEAVY CLASS IMBALANCE DETECTED - will apply special handling")

#  Define column groups
text_cols = ['title', 'company_profile', 'description', 'requirements', 'benefits']
cat_cols = ['location', 'department', 'employment_type', 'required_experience',
            'required_education', 'industry', 'function']
binary_cols = ['telecommuting', 'has_company_logo', 'has_questions']

#  Fill missing values
print("\n" + "-"*60)
print("Handling Missing Values")
print("-"*60)

# Text columns - empty string
for col in text_cols:
    missing_count = df[col].isna().sum()
    df[col] = df[col].fillna('')
    if missing_count > 0:
        print(f"  {col}: {missing_count} missing values filled")

# Categorical columns - 'Unknown' category
for col in cat_cols:
    missing_count = df[col].isna().sum()
    df[col] = df[col].fillna('Unknown')
    if missing_count > 0:
        print(f"  {col}: {missing_count} missing -> 'Unknown'")

# Binary columns - mode
for col in binary_cols:
    missing_count = df[col].isna().sum()
    if missing_count > 0:
        mode_val = df[col].mode()[0]
        df[col] = df[col].fillna(mode_val)
        print(f"  {col}: {missing_count} missing -> {mode_val}")

# Salary - binary indicator
df['salary_specified'] = df['salary_range'].apply(
    lambda x: 0 if pd.isna(x) or str(x).strip()=="" else 1
)
print(f"  salary_specified: {df['salary_specified'].sum()} have salary info")
df.drop(columns=['salary_range'], inplace=True)

# 4. Reduce categorical cardinality
print("\n" + "-"*60)
print("Reducing Categorical Cardinality")
print("-"*60)

def reduce_cardinality(series, threshold=50, other_label='Other'):
    """Reduce high cardinality by keeping top N categories"""
    value_counts = series.value_counts()

    mask = series.isin(value_counts[value_counts >= threshold].index)
    reduced_series = series.where(mask, other_label)

    original_unique = series.nunique()
    new_unique = reduced_series.nunique()

    print(f"  {series.name}: {original_unique} → {new_unique} categories")
    return reduced_series

# Apply to high-cardinality columns
df['location'] = reduce_cardinality(df['location'], threshold=100)
df['department'] = reduce_cardinality(df['department'], threshold=50)
df['industry'] = reduce_cardinality(df['industry'], threshold=30)

#  Combine and clean text
print("\n" + "-"*60)
print("Text Processing")
print("-"*60)

df['text_raw'] = df[text_cols].agg(' '.join, axis=1)
print(f"  Raw text combined from {len(text_cols)} columns")

# Clean text
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):

    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()


    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()


    words = [word for word in text.split() if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

df['text_clean'] = df['text_raw'].apply(clean_text)

# Handle empty text
empty_count = (df['text_clean'].str.strip() == '').sum()
df['text_clean'] = df['text_clean'].apply(lambda x: x if x.strip() != '' else 'emptytext')
print(f"  Empty text documents: {empty_count}")




print("\n" + "-"*60)
print("Feature Engineering")
print("-"*60)

def extract_text_features(row):
    raw = row['text_raw']
    clean = row['text_clean']
    words = clean.split()
    wc = len(words)

    uppercase_words = sum(1 for word in raw.split() if word.isupper() and len(word) > 1)

    return pd.Series({
        'char_count': len(raw),
        'word_count': wc,
        'unique_words': len(set(words)),
        'avg_word_len': (sum(len(w) for w in words)/wc) if wc > 0 else 0,
        'num_exclaims': raw.count('!'),
        'num_questions': raw.count('?'),
        'has_email': 1 if '@' in raw else 0,
        'has_url': 1 if 'http' in raw.lower() or 'www' in raw.lower() else 0,
        'all_caps_ratio': sum(1 for c in raw if c.isupper()) / len(raw) if len(raw) > 0 else 0,
        'uppercase_word_count': uppercase_words,
        'text_richness': len(set(words)) / wc if wc > 0 else 0
    })

numeric_text_cols = ['char_count', 'word_count', 'unique_words', 'avg_word_len',
                     'num_exclaims', 'num_questions', 'has_email', 'has_url',
                     'all_caps_ratio', 'uppercase_word_count', 'text_richness']

df[numeric_text_cols] = df.apply(extract_text_features, axis=1)
print(f"  Extracted {len(numeric_text_cols)} numeric text features")


print("\n  Feature correlation with fraud:")
fraud_correlations = {}
for col in numeric_text_cols:
    corr = df[col].corr(df['fraudulent'])
    fraud_correlations[col] = corr
    print(f"    {col:.<20} {corr:7.4f}")


#  Encode categorical features

print("\n" + "-"*60)
print("Categorical Encoding")
print("-"*60)

for col in cat_cols:
    unique_count = df[col].nunique()
    print(f"  {col}: {unique_count} unique values")

df_cat_encoded = pd.get_dummies(df[cat_cols], drop_first=True)
print(f"\n  One-hot encoded to {df_cat_encoded.shape[1]} features (reduced from 4634+)")


# 8. TF-IDF features

print("TF-IDF Vectorization")


tfidf_vectorizer = TfidfVectorizer(
    max_features=2000,
    min_df=10,
    max_df=0.8,
    ngram_range=(1, 2),
    stop_words='english'
)

X_text = tfidf_vectorizer.fit_transform(df['text_clean'])
print(f"  TF-IDF shape: {X_text.shape}")
print(f"  Sparsity: {(1 - X_text.nnz / (X_text.shape[0] * X_text.shape[1])) * 100:.2f}%")

#  Scale  features
print("Feature Scaling")
# Separate continuous and binary features
continuous_features = ['char_count', 'word_count', 'unique_words', 'avg_word_len',
                      'all_caps_ratio', 'uppercase_word_count', 'text_richness']
binary_features = ['num_exclaims', 'num_questions', 'has_email', 'has_url'] + binary_cols + ['salary_specified']

scaler = StandardScaler()
X_continuous = scaler.fit_transform(df[continuous_features])
print(f"  Scaled {len(continuous_features)} continuous features")

X_binary = df[binary_features].values
print(f"  Kept {len(binary_features)} binary features unscaled")

X_num = np.hstack([X_continuous, X_binary])
print(f"  Total numeric features: {X_num.shape[1]}")

#  Combine all features

print("\n" + "-"*60)
print("Combining Features")
print("-"*60)

X_combined = sparse.hstack([
    X_text,
    sparse.csr_matrix(X_num),
    sparse.csr_matrix(df_cat_encoded.values)
], format='csr')

print(f"  Initial feature matrix: {X_combined.shape}")


#  FEATURE SELECTION

print("Feature Selection for Imbalanced Data")


# Calculate optimal number of features (rule of thumb for imbalanced data)
n_fraud = df['fraudulent'].sum()
optimal_features = min(500, n_fraud * 2)
optimal_features = max(100, optimal_features)

print(f"  Fraud samples: {n_fraud}")
print(f"  Optimal features (2× fraud samples): {optimal_features}")

print("  Using ANOVA F-value for feature selection (handles negative values)")
selector = SelectKBest(f_classif, k=optimal_features)
X_final = selector.fit_transform(X_combined, df['fraudulent'])

print(f"  Reduced from {X_combined.shape[1]} to {X_final.shape[1]} features")
print(f"  Feature reduction: {(1 - X_final.shape[1] / X_combined.shape[1]) * 100:.1f}%")


print("Data Quality Checks")


y = df['fraudulent']

print(f"  Final dataset shape: {X_final.shape}")
print(f"  Fraud ratio: {y.mean()*100:.2f}%")
print(f"  Fraud samples: {y.sum()}")
print(f"  Legitimate samples: {len(y) - y.sum()}")
print(f"  Feature sparsity: {(1 - X_final.nnz / (X_final.shape[0] * X_final.shape[1])) * 100:.2f}%")

if X_final.shape[0] != len(y):
    print("  ⚠ WARNING: Feature matrix and target length mismatch!")
if sparse.issparse(X_final):
    if np.any(np.isnan(X_final.data)):
        print("  ⚠ WARNING: NaN values detected in features!")
if X_final.nnz == 0:
    print("  ⚠ WARNING: Feature matrix is all zeros!")

print("  ✓ All quality checks passed")


#  Save preprocessed outputs

print("\n" + "-"*60)
print("Saving Outputs")
print("-"*60)

sparse.save_npz("X_processed.npz", X_final)
print("  ✓ X_processed.npz saved")

with open("y_target.pkl", "wb") as f:
    pickle.dump(y, f)
print("  ✓ y_target.pkl saved")

with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)
print("  ✓ tfidf_vectorizer.pkl saved")

with open("feature_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
print("  ✓ feature_scaler.pkl saved")

with open("feature_selector.pkl", "wb") as f:
    pickle.dump(selector, f)
print("  ✓ feature_selector.pkl saved")

feature_names = {
    'tfidf_features': tfidf_vectorizer.get_feature_names_out().tolist(),
    'numeric_features': continuous_features + binary_features,
    'categorical_features': df_cat_encoded.columns.tolist(),
    'selected_features_mask': selector.get_support()
}

with open("feature_names.pkl", "wb") as f:
    pickle.dump(feature_names, f)
print("  ✓ feature_names.pkl saved")

df.to_csv("processed_job_postings.csv", index=False)
print("  ✓ processed_job_postings.csv saved")

print("\n" + "="*60)
print("PREPROCESSING COMPLETED SUCCESSFULLY!")
print("="*60)
print(f"\nFinal optimized dataset:")
print(f"  Samples: {X_final.shape[0]}")
print(f"  Features: {X_final.shape[1]} (reduced for imbalance)")
print(f"  Fraud ratio: {y.mean()*100:.2f}%")
print(f"  Ready for imbalanced learning techniques! 🚀")



Manual Perceptron

In [ ]:
import numpy as np
import pickle
import json
import os
from scipy import sparse
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight


os.makedirs('models', exist_ok=True)

print("\n[1] Loading and preparing data...")

X_processed = sparse.load_npz("X_processed.npz")
with open("y_target.pkl", "rb") as f:
    y = pickle.load(f)

print(f"Dataset: {X_processed.shape}")
print(f"Fraud ratio: {y.mean()*100:.4f}% ({y.sum()} frauds)")

X_dense = X_processed.toarray()
if not isinstance(y, np.ndarray):
    y = y.values

X_with_bias = np.hstack([np.ones((X_dense.shape[0], 1)), X_dense])

X_temp, X_test, y_temp, y_test = train_test_split(X_with_bias, y, test_size=0.1, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1111, random_state=42, stratify=y_temp)

print("\nData shapes:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape} ({y_train.sum()} frauds)")
print(f"X_val:   {X_val.shape}, y_val: {y_val.shape} ({y_val.sum()} frauds)")
print(f"X_test:  {X_test.shape}, y_test: {y_test.shape} ({y_test.sum()} frauds)")

class ManualPerceptron:
    def _init_(self, learning_rate=0.01, n_iters=1000, class_weight=None):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.class_weight = class_weight
        self.weights = None
        self.errors_history = []
        self.loss_history = []

    def _calculate_sample_weights(self, y):
        if self.class_weight == 'balanced':
            class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
            sample_weights = np.where(y == 1, class_weights[1], class_weights[0])
        elif isinstance(self.class_weight, dict):
            sample_weights = np.where(y == 1, self.class_weight[1], self.class_weight[0])
        else:
            sample_weights = np.ones(len(y))
        return sample_weights

    def activation(self, x):
        return np.where(x >= 0, 1, 0)

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.random.normal(0, 0.01, n_features)
        sample_weights = self._calculate_sample_weights(y)

        for epoch in range(self.n_iters):
            total_errors = 0
            total_loss = 0

            for i in range(n_samples):
                linear_output = np.dot(X[i], self.weights)
                y_pred = self.activation(linear_output)

                error = (y[i] - y_pred) * sample_weights[i]
                total_loss += abs(error)

                if error != 0:
                    self.weights += self.lr * error * X[i]
                    total_errors += 1

            self.errors_history.append(total_errors)
            self.loss_history.append(total_loss / n_samples)

            if (epoch + 1) % 100 == 0:
                print(f"    Epoch {epoch + 1}: Errors = {total_errors}, Loss = {self.loss_history[-1]:.4f}")

            if total_errors == 0:
                print(f"    Converged after {epoch + 1} epochs")
                break

    def predict(self, X):
        linear_output = np.dot(X, self.weights)
        return self.activation(linear_output)

    def predict_proba(self, X):
        linear_output = np.dot(X, self.weights)
        probabilities = 1 / (1 + np.exp(-linear_output))
        return probabilities

    def decision_function(self, X):
        return np.dot(X, self.weights)

print("\n[2] Hyperparameter tuning with class weights...")

n_legit = (y_train == 0).sum()
n_fraud = (y_train == 1).sum()
inverse_freq = n_legit / n_fraud

print(f"  Class distribution: {n_legit} legitimate, {n_fraud} fraudulent")
print(f"  Inverse frequency weight: {inverse_freq:.2f}")

learning_rates = [0.1, 0.01, 0.001]
weight_strategies = [
    None,
    'balanced',
    {0: 1, 1: 10},
    {0: 1, 1: 20}
]

best_f1 = -1
best_config = {}
results = {}

for lr in learning_rates:
    for weights in weight_strategies:
        config_name = f"lr_{lr}weights{weights}"
        print(f"\n  Testing: {config_name}")

        perceptron = ManualPerceptron(learning_rate=lr, n_iters=500, class_weight=weights)
        perceptron.fit(X_train, y_train)

        y_val_pred = perceptron.predict(X_val)
        f1 = f1_score(y_val, y_val_pred)
        recall = recall_score(y_val, y_val_pred)

        results[config_name] = {
            'f1': f1,
            'recall': recall,
            'model': perceptron
        }

        print(f"    Validation F1: {f1:.4f}, Recall: {recall:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            best_config = {'lr': lr, 'weights': weights}
            best_model = perceptron

print(f"\nBest configuration: LR={best_config['lr']}, Weights={best_config['weights']}")
print(f"Best Validation F1-Score: {best_f1:.4f}")

print("\n[3] Final evaluation on test set...")

y_test_pred = best_model.predict(X_test)
y_test_proba = best_model.predict_proba(X_test)

accuracy = float(accuracy_score(y_test, y_test_pred))
precision = float(precision_score(y_test, y_test_pred))
recall = float(recall_score(y_test, y_test_pred))
f1 = float(f1_score(y_test, y_test_pred))
roc_auc = float(roc_auc_score(y_test, y_test_proba))
pr_auc = float(average_precision_score(y_test, y_test_proba))

cm = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cm.ravel()

print("\nPERCEPTRON TEST RESULTS")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print(f"PR-AUC:    {pr_auc:.4f}")

print(f"\nConfusion Matrix:")
print(f"                Predicted")
print(f"               Legit   Fraud")
print(f"Actual Legit   {tn:>5}   {fp:>5}")
print(f"Actual Fraud   {fn:>5}   {tp:>5}")

print(f"\nBusiness Impact:")
print(f"Frauds caught: {tp}/{tp+fn} ({recall*100:.1f}%)")
print(f"Frauds missed: {fn}")
print(f"False alarms:  {fp} legitimate jobs flagged")

print(f"\nClassification Report:")
print(classification_report(y_test, y_test_pred, target_names=['Legitimate', 'Fraud']))

print("\n[4] Saving Manual Perceptron model with comprehensive metrics...")

def save_perceptron_model_with_metrics(model, metrics, model_name, config):
    model_filename = f"models/{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_model.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

    metrics_filename = f"models/{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_metrics.json"
    metrics_to_save = {
        'model_name': model_name,
        'model_type': "ManualPerceptron",
        'optimal_threshold': 0.0,
        'performance_metrics': {
            'recall': float(metrics['recall']),
            'f1_score': float(metrics['f1']),
            'accuracy': float(metrics['accuracy']),
            'auc_roc': float(metrics['roc_auc']),
            'auc_pr': float(metrics['pr_auc'])
        },
        'confusion_matrix': {
            'tn': int(tn),
            'fp': int(fp),
            'fn': int(fn),
            'tp': int(tp)
        },
        'business_metrics': {
            'fraud_detection_rate': float(tp/(tp+fn)*100) if (tp+fn) > 0 else 0.0,
            'false_alarm_rate': float(fp/(fp+tn)*100) if (fp+tn) > 0 else 0.0,
            'frauds_caught': int(tp),
            'frauds_missed': int(fn),
            'false_positives': int(fp)
        },
        'training_info': {
            'train_samples': int(X_train.shape[0]),
            'val_samples': int(X_val.shape[0]),
            'test_samples': int(X_test.shape[0]),
            'fraud_ratio': float(y.mean()),
            'learning_rate': float(config['lr']),
            'class_weight': config['weights'],
            'n_features': int(X_dense.shape[1])
        },
        'model_config': {
            'learning_rate': float(config['lr']),
            'class_weight': config['weights'],
            'n_iterations': len(model.errors_history)
        }
    }

    with open(metrics_filename, 'w') as f:
        json.dump(metrics_to_save, f, indent=2)

    return model_filename, metrics_filename

perceptron_metrics = {
    'precision': precision,
    'recall': recall,
    'f1': f1,
    'accuracy': accuracy,
    'roc_auc': roc_auc,
    'pr_auc': pr_auc
}

best_model_file, best_metrics_file = save_perceptron_model_with_metrics(
    best_model, perceptron_metrics, "Manual Perceptron", best_config
)

print(f"  {best_model_file}")
print(f"  {best_metrics_file}")

perceptron_package = {
    'best_model': best_model,
    'performance_metrics': perceptron_metrics,
    'confusion_matrix': {
        'tn': int(tn), 'fp': int(fp), 'fn': int(fn), 'tp': int(tp)
    },
    'auc_scores': {
        'roc_auc': float(roc_auc),
        'pr_auc': float(pr_auc)
    },
    'training_info': {
        'dataset_shape': [int(X_dense.shape[0]), int(X_dense.shape[1])],
        'fraud_ratio': float(y.mean()),
        'train_samples': int(X_train.shape[0]),
        'test_samples': int(X_test.shape[0]),
        'best_config': best_config
    }
}

with open("models/manual_perceptron_model.pkl", "wb") as f:
    pickle.dump(perceptron_package, f)

with open("models/manual_perceptron_metrics.json", "w") as f:
    json.dump({
        'best_model': "Manual Perceptron",
        'performance_metrics': perceptron_metrics,
        'business_impact': {
            'frauds_caught': f"{tp}/{tp+fn} ({tp/(tp+fn)*100:.1f}%)",
            'frauds_missed': int(fn),
            'false_alarms': int(fp),
            'fraud_detection_rate': float(tp/(tp+fn)*100) if (tp+fn) > 0 else 0.0,
            'false_alarm_rate': float(fp/(fp+tn)*100) if (fp+tn) > 0 else 0.0
        },
        'best_configuration': best_config
    }, f, indent=2)

print(f"  models/manual_perceptron_model.pkl")
print(f"  models/manual_perceptron_metrics.json")

print("\n[5] Generating visualizations...")

try:
    import seaborn as sns
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    axes[0,0].plot(best_model.errors_history, marker='o', color='blue', linewidth=2)
    axes[0,0].set_title(f'Training Errors (LR={best_config["lr"]})')
    axes[0,0].set_xlabel('Epoch')
    axes[0,0].set_ylabel('Misclassifications')
    axes[0,0].grid(True, alpha=0.3)

    axes[0,1].plot(best_model.loss_history, marker='s', color='red', linewidth=2)
    axes[0,1].set_title('Training Loss')
    axes[0,1].set_xlabel('Epoch')
    axes[0,1].set_ylabel('Average Loss')
    axes[0,1].grid(True, alpha=0.3)

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1,0],
                xticklabels=['Legitimate', 'Fraud'],
                yticklabels=['Legitimate', 'Fraud'])
    axes[1,0].set_title('Confusion Matrix')
    axes[1,0].set_xlabel('Predicted')
    axes[1,0].set_ylabel('Actual')

    legit_scores = y_test_proba[y_test == 0]
    fraud_scores = y_test_proba[y_test == 1]
    axes[1,1].hist(legit_scores, bins=30, alpha=0.7, label='Legitimate', color='green', density=True)
    axes[1,1].hist(fraud_scores, bins=30, alpha=0.7, label='Fraud', color='red', density=True)
    axes[1,1].axvline(0.5, color='black', linestyle='--', label='Decision Boundary')
    axes[1,1].set_xlabel('Prediction Score')
    axes[1,1].set_ylabel('Density')
    axes[1,1].set_title('Score Distribution by Class')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('models/perceptron_results.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Visualizations saved as 'models/perceptron_results.png'")
except Exception as e:
    print(f"Visualization generation skipped: {e}")

print("\nMANUAL PERCEPTRON TRAINING COMPLETED!")

print(f"\nFINAL PERFORMANCE:")
print(f"  F1-Score: {f1:.4f}")
print(f"  Recall:   {recall:.4f} ({recall*100:.1f}% frauds caught)")
print(f"  Precision: {precision:.4f} ({precision*100:.1f}% accurate when flagging)")
print(f"  ROC-AUC:   {roc_auc:.4f}")
print(f"  PR-AUC:    {pr_auc:.4f}")

print(f"\nBEST CONFIGURATION:")
print(f"  Learning Rate: {best_config['lr']}")
print(f"  Class Weights: {best_config['weights']}")

print(f"\nDATASET INFO:")
print(f"  Total samples: {X_dense.shape[0]}")
print(f"  Features: {X_dense.shape[1]}")
print(f"  Fraud rate: {y.mean()*100:.2f}%")

print(f"\nModels saved in 'models/' directory:")
print(f"  Manual Perceptron: models/manual_perceptron_model.pkl")
print(f"  Metrics: models/manual_perceptron_metrics.json")

print(f"\nManual Perceptron is ready for production deployment!")

Neural Network

In [ ]:
# Neural Network

import numpy as np
import pandas as pd
from scipy import sparse
import pickle
import json
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.decomposition import TruncatedSVD

print("Loading processed data...")
X_sparse = sparse.load_npz("X_processed.npz")

with open("y_target.pkl", "rb") as f:
    y = pickle.load(f)
y = np.array(y).ravel()

print(f"Loaded X: {X_sparse.shape}, y: {len(y)}")

# APPLY SVD → 500 DIMENSIONS
print("Applying TruncatedSVD (500 components)...")
svd = TruncatedSVD(n_components=500, random_state=42)
X_svd = svd.fit_transform(X_sparse)

print(f"SVD reduced shape: {X_svd.shape}")

# TRAIN/VAL/TEST SPLIT
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_svd, y, test_size=0.2, stratify=y, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.15, stratify=y_train_val, random_state=42
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# MANUAL NEURAL NETWORK CLASS
class ManualL3NN:
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], output_dim=1):
        np.random.seed(42)
        self.params = {}
        dims = [input_dim] + hidden_dims + [output_dim]

        for i in range(1, len(dims)):
            self.params[f"W{i}"] = np.random.randn(dims[i-1], dims[i]) * np.sqrt(2.0 / dims[i-1])
            self.params[f"b{i}"] = np.zeros((1, dims[i]))
            if i < len(dims) - 1:
                self.params[f"gamma{i}"] = np.ones((1, dims[i]))
                self.params[f"beta{i}"] = np.zeros((1, dims[i]))

        self.bn_params = {
            i: {"running_mean": np.zeros((1, hidden_dims[i-1])),
                "running_var": np.ones((1, hidden_dims[i-1]))}
            for i in range(1, len(hidden_dims)+1)
        }

        self.m, self.v, self.t = {}, {}, 0
        for k in self.params:
            self.m[k] = np.zeros_like(self.params[k])
            self.v[k] = np.zeros_like(self.params[k])

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -20, 20)))

    def relu(self, z):
        return np.maximum(0, z)

    def batchnorm_forward(self, z, layer, mode="train"):
        gamma = self.params[f"gamma{layer}"]
        beta = self.params[f"beta{layer}"]
        bn = self.bn_params[layer]

        if mode == "train":
            mu = np.mean(z, axis=0, keepdims=True)
            var = np.var(z, axis=0, keepdims=True)
            var = np.clip(var, 1e-8, None)
            z_hat = (z - mu) / np.sqrt(var + 1e-8)
            out = gamma * z_hat + beta
            bn["running_mean"] = 0.9 * bn["running_mean"] + 0.1 * mu
            bn["running_var"] = 0.9 * bn["running_var"] + 0.1 * var
            return out, (z, z_hat, mu, var, gamma, beta)
        else:
            z_hat = (z - bn["running_mean"]) / np.sqrt(bn["running_var"] + 1e-8)
            return gamma * z_hat + beta, None

    def batchnorm_backward(self, dout, cache):
        z, z_hat, mu, var, gamma, beta = cache
        N = z.shape[0]
        dbeta = np.sum(dout, axis=0, keepdims=True)
        dgamma = np.sum(dout * z_hat, axis=0, keepdims=True)
        dz_hat = dout * gamma
        dvar = np.sum(dz_hat * (z - mu) * -0.5 * (var + 1e-8)**(-1.5), axis=0, keepdims=True)
        dmu = np.sum(dz_hat * -1/np.sqrt(var + 1e-8), axis=0, keepdims=True) + dvar * np.mean(-2*(z - mu), axis=0, keepdims=True)
        dz = dz_hat / np.sqrt(var + 1e-8) + dvar * 2*(z - mu)/N + dmu/N
        return dz, dgamma, dbeta

    def forward(self, X, mode="train"):
        caches = {}
        A = X.astype(np.float32)
        activations = [A]

        for i in range(1, 4):
            Z = A @ self.params[f"W{i}"] + self.params[f"b{i}"]
            Z_bn, bn_cache = self.batchnorm_forward(Z, i, mode)
            A = self.relu(Z_bn)
            activations.append(A)
            caches[i] = (Z, Z_bn, bn_cache)

        Z_out = A @ self.params["W4"] + self.params["b4"]
        y_hat = self.sigmoid(Z_out)
        caches["activations"] = activations
        return y_hat, caches

    def backward(self, y_true, y_pred, caches):
        grads = {}
        m = y_true.shape[0]
        dZ = y_pred - y_true

        A3 = caches["activations"][3]
        grads["W4"] = A3.T @ dZ / m
        grads["b4"] = np.sum(dZ, axis=0, keepdims=True) / m
        dA = dZ @ self.params["W4"].T

        for i in reversed(range(1, 4)):
            Z, Z_bn, bn_cache = caches[i]
            dZ_bn = dA * (Z_bn > 0)
            dZ, dgamma, dbeta = self.batchnorm_backward(dZ_bn, bn_cache)
            grads[f"gamma{i}"] = dgamma
            grads[f"beta{i}"] = dbeta

            A_prev = caches["activations"][i-1]
            grads[f"W{i}"] = A_prev.T @ dZ / m
            grads[f"b{i}"] = np.sum(dZ, axis=0, keepdims=True) / m
            dA = dZ @ self.params[f"W{i}"].T

        return grads

    def update_params(self, grads, lr=1e-3):
        self.t += 1
        for k in self.params:
            g = grads[k]
            self.m[k] = 0.9 * self.m[k] + 0.1 * g
            self.v[k] = 0.999 * self.v[k] + 0.001 * (g ** 2)
            m_hat = self.m[k] / (1 - 0.9 ** self.t)
            v_hat = self.v[k] / (1 - 0.999 ** self.t)
            self.params[k] -= lr * m_hat / (np.sqrt(v_hat) + 1e-8)

    def fit(self, X_train, y_train, X_val, y_val, epochs=60, lr=1e-3, batch_size=128):
        y_train = y_train.reshape(-1, 1)
        y_val = y_val.reshape(-1, 1)

        for epoch in range(1, epochs + 1):
            perm = np.random.permutation(len(X_train))
            X_shuf = X_train[perm]
            y_shuf = y_train[perm]

            for i in range(0, len(X_train), batch_size):
                Xb = X_shuf[i:i+batch_size]
                yb = y_shuf[i:i+batch_size]
                y_pred, caches = self.forward(Xb, "train")
                grads = self.backward(yb, y_pred, caches)
                self.update_params(grads, lr)

            if epoch % 10 == 0 or epoch == epochs:
                val_pred, _ = self.forward(X_val, "eval")
                val_f1 = f1_score(y_val, (val_pred >= 0.5).astype(int))
                print(f"Epoch {epoch:2d} | Val F1: {val_f1:.4f}")

    def predict_proba(self, X):
        y_pred, _ = self.forward(X, "eval")
        return y_pred.ravel()

    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)


# TRAIN THE MODEL
print("\nTraining Manual Neural Network (500 → 256 → 128 → 64 → 1)...")
model = ManualL3NN(input_dim=500)
model.fit(X_train, y_train, X_val, y_val, epochs=60, lr=1e-3, batch_size=128)

# EVALUATE ON TEST SET
print("\nEvaluating on test set...")
probs = model.predict_proba(X_test)
preds = model.predict(X_test)

metrics = {
    "accuracy": float(accuracy_score(y_test, preds)),
    "precision": float(precision_score(y_test, preds)),
    "recall": float(recall_score(y_test, preds)),
    "f1": float(f1_score(y_test, preds)),
    "auc": float(roc_auc_score(y_test, probs))
}

for k, v in metrics.items():
    print(f"{k.capitalize():9}: {v:.4f}")

os.makedirs("models", exist_ok=True)
os.makedirs("metrics", exist_ok=True)

# Save model
with open("models/manual_nn.pkl", "wb") as f:
    pickle.dump(model, f)

# Save SVD (CRITICAL!)
with open("models/svd_500_component.pkl", "wb") as f:
    pickle.dump(svd, f)

# Save metrics
with open("metrics/manual_nn_metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)

print("\nSAVED:")
print("   models/manual_nn.pkl")
print("   models/svd_500_component.pkl   ← MUST UPLOAD TO STREAMLIT!")
print("   metrics/manual_nn_metrics.json")
print("\nDone! Your Streamlit app will now work perfectly.")

KNN

In [ ]:
# KNN IMPLEMENTATION
import numpy as np
import pandas as pd
import pickle, json, os
from scipy import sparse
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)


X = sparse.load_npz("X_processed.npz")
y = pickle.load(open("y_target.pkl", "rb"))

print(f"Loaded X: {X.shape}, y: {len(y)}")
print(f"Fraud ratio: {y.mean()*100:.2f}%")



X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)



def normalize_rows(x):
    norms = np.linalg.norm(x, axis=1, keepdims=True) + 1e-10
    return x / norms


class ManualKNN:
    def __init__(self, n_neighbors=3, p=1, weights="distance"):
        self.k = n_neighbors
        self.p = p
        self.weights = weights

    def fit(self, X, y):
        self.X_train = np.asarray(X)
        self.y_train = np.asarray(y).astype(int)

    def compute_distances(self, x):
        return np.sum(np.abs(self.X_train - x), axis=1)

    def predict_one(self, x):
        distances = self.compute_distances(x)

        idx = np.argsort(distances)[:self.k]
        labels = self.y_train[idx]
        dists = distances[idx]

        weights = 1 / (dists + 1e-10)

        score1 = np.sum(weights * labels)
        score0 = np.sum(weights * (1 - labels))

        return 1 if score1 > score0 else 0

    def predict(self, X):
        X = np.asarray(X)
        return np.array([self.predict_one(row) for row in X])

    def predict_proba(self, X):
        X = np.asarray(X)
        probs = []
        for row in X:
            distances = self.compute_distances(row)
            idx = np.argsort(distances)[:self.k]
            labels = self.y_train[idx]
            dists = distances[idx]
            weights = 1 / (dists + 1e-10)
            p1 = np.sum(weights * labels)
            p0 = np.sum(weights * (1 - labels))
            probs.append([p0/(p0+p1), p1/(p0+p1)])
        return np.array(probs)




# SVD DIMENSIONALITY REDUCTION FIRST
svd = TruncatedSVD(n_components=300, random_state=42)
X_train_svd = svd.fit_transform(X_train)
X_test_svd = svd.transform(X_test)

# Normalize manually
X_train_norm = normalize_rows(X_train_svd)
X_test_norm = normalize_rows(X_test_svd)



knn = ManualKNN(n_neighbors=3, p=1, weights="distance")
knn.fit(X_train_norm, y_train)

y_pred = knn.predict(X_test_norm)
y_prob = knn.predict_proba(X_test_norm)[:, 1]

metrics = {
    "accuracy": float(accuracy_score(y_test, y_pred)),
    "precision": float(precision_score(y_test, y_pred)),
    "recall": float(recall_score(y_test, y_pred)),
    "f1": float(f1_score(y_test, y_pred)),
    "auc": float(roc_auc_score(y_test, y_prob)),
    "best_params": {
        "n_neighbors": 3,
        "p": 1,
        "weights": "distance"
    }
}

print("\n✅ Test Metrics:")
for k, v in metrics.items():
    if k != "best_params":
        print(f"{k}: {v:.4f}")




os.makedirs("metrics", exist_ok=True)
json.dump(metrics, open("metrics/knn_metrics.json", "w"), indent=4)

print("✅ Saved metrics → metrics/knn_metrics.json")


# SAVE MANUAL KNN MODEL
os.makedirs("models", exist_ok=True)

save_obj = {
    "svd": svd,
    "knn": knn
}

pickle.dump(save_obj, open("models/best_knn_model.pkl", "wb"))

print(" Saved Manual KNN → models/best_knn_model.pkl")
print("\n Manual KNN Training Complete!")

Probabilistic Models

In [ ]:
# PROBABILISTIC MODELS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle, os, warnings, json
from scipy import sparse
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, confusion_matrix, roc_curve, auc)
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings("ignore")
np.random.seed(42)

# LOAD PREPROCESSED DATA
print("🔹 Loading preprocessed data...")

X_combined = sparse.load_npz("X_processed.npz")
with open("y_target.pkl", "rb") as f:
    y = pickle.load(f)

print(" Data loaded successfully.")
print("Feature matrix shape:", X_combined.shape)
print("Target shape:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=42, stratify=y
)

# ENSURE NON-NEGATIVE VALUES FOR NB MODELS
X_train_nb = X_train.copy()
X_test_nb = X_test.copy()
X_train_nb.data = np.clip(X_train_nb.data, 0, None)
X_test_nb.data = np.clip(X_test_nb.data, 0, None)

# EVALUATION FUNCTION
def evaluate_model(model, X_train, X_test, y_train, y_test, name):
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    y_proba_test = model.predict_proba(X_test)[:, 1]

    metrics = {
        "Model": name,
        "Train Accuracy": float(accuracy_score(y_train, y_pred_train)),
        "Test Accuracy": float(accuracy_score(y_test, y_pred_test)),
        "Precision": float(precision_score(y_test, y_pred_test)),
        "Recall": float(recall_score(y_test, y_pred_test)),
        "F1-Score": float(f1_score(y_test, y_pred_test))
    }

    cm = confusion_matrix(y_test, y_pred_test).tolist()  # convert to list for JSON
    fpr, tpr, _ = roc_curve(y_test, y_proba_test)
    roc_auc = float(auc(fpr, tpr))
    metrics["ROC-AUC"] = roc_auc

    return metrics, cm, (fpr.tolist(), tpr.tolist(), roc_auc)

# 5. MODEL SETUP WITH PARAM GRIDS
models = {
    "MultinomialNB": (MultinomialNB(), {"alpha": [0.1, 0.5, 1.0, 2.0]}),
    "ComplementNB": (ComplementNB(), {"alpha": [0.1, 0.5, 1.0, 2.0]}),
    "BernoulliNB": (BernoulliNB(), {"alpha": [0.1, 0.5, 1.0, 2.0], "binarize": [0.0, 0.1, 0.5]}),
    "LogisticRegression": (LogisticRegression(max_iter=1000, solver="liblinear"), {"C": [0.1, 1, 10]})
}

#  HYPERPARAMETER TUNING & EVALUATION
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
all_metrics, all_cms, all_rocs = [], {}, {}
best_models = {}

for name, (model, params) in models.items():
    print(f"\n Tuning {name}...")

    Xtr, Xte = (X_train_nb, X_test_nb) if "NB" in name else (X_train, X_test)
    grid = GridSearchCV(model, param_grid=params, scoring="f1", cv=cv, n_jobs=-1, verbose=1)
    grid.fit(Xtr, y_train)
    best_model = grid.best_estimator_
    best_models[name] = best_model

    print(f" Best Params for {name}: {grid.best_params_}")

    metrics, cm, roc_data = evaluate_model(best_model, Xtr, Xte, y_train, y_test, name)
    all_metrics.append(metrics)
    all_cms[name] = cm
    all_rocs[name] = roc_data

metrics_df = pd.DataFrame(all_metrics)
print("\n=== Final Model Performance ===")
print(metrics_df)


os.makedirs("saved_models", exist_ok=True)

# Save models as .pkl
for name, model in best_models.items():
    with open(f"saved_models/{name}_best.pkl", "wb") as f:
        pickle.dump(model, f)

# Save metrics as JSON
metrics_json = {
    "all_metrics": all_metrics,
    "confusion_matrices": all_cms,
    "roc_data": all_rocs
}
with open("saved_models/metrics.json", "w") as f:
    json.dump(metrics_json, f, indent=4)

print("\n All models and metrics saved successfully in 'saved_models/' folder.")

# VISUALIZATION
plt.figure(figsize=(10, 7))
for name, (fpr, tpr, roc_auc) in all_rocs.items():
    plt.plot(fpr, tpr, label=f"{name} (AUC={roc_auc:.2f})")
plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for Probabilistic Models")
plt.legend()
plt.savefig("saved_models/roc_curves.png")
plt.show()

for name, cm in all_cms.items():
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.savefig(f"saved_models/{name}_cm.png")
    plt.show()

metrics_df.set_index("Model", inplace=True)
metrics_df.plot(kind="bar", figsize=(10, 6))
plt.title("Model Performance Comparison")
plt.ylabel("Score")
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("saved_models/model_comparison.png")
plt.show()

#  BEST MODEL SELECTION
best_idx = metrics_df["F1-Score"].idxmax()
best_row = metrics_df.loc[best_idx]

print("\n" + "="*60)
print(" BEST MODEL BASED ON F1-SCORE")
print("="*60)
print(best_row)
print(f"\n Recommended Model: {best_idx}")

# Save best model name
with open("saved_models/best_model_name.json", "w") as f:
    json.dump({"best_model": best_idx}, f, indent=4)


SVM

In [ ]:
import numpy as np
import pickle
import json
import os
from scipy import sparse
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_recall_curve, average_precision_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')



os.makedirs('models', exist_ok=True)

# 1. Load Preprocessed Data
print("\n[1/8] Loading optimized preprocessed dataset...")

X_processed = sparse.load_npz("X_processed.npz")
with open("y_target.pkl", "rb") as f:
    y = pickle.load(f)

print(f"✓ Dataset: {X_processed.shape}")
print(f"✓ Fraud ratio: {y.mean()*100:.4f}% ({y.sum()} frauds out of {len(y)})")

# Convert to dense for better SVM performance
X_dense = X_processed.toarray()
print(f"✓ Converted to dense format: {X_dense.shape}")


# STRATIFIED TRAIN-TEST SPLIT
print("\n[2/8] Creating stratified split...")

X_train, X_test, y_train, y_test = train_test_split(
    X_dense, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(f" Training: {X_train.shape} - {y_train.sum()} frauds ({y_train.mean()*100:.4f}%)")
print(f" Testing:  {X_test.shape} - {y_test.sum()} frauds ({y_test.mean()*100:.4f}%)")


# CLASS WEIGHT CALCULATION

print("\n[3/8] Calculating optimal class weights...")

#  Standard balanced weights
standard_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
standard_weight_dict = {0: float(standard_weights[0]), 1: float(standard_weights[1])}

#  Custom weights
cost_missed_fraud = 10
cost_false_alarm = 1
custom_weights = {
    0: 1.0,
    1: float((len(y_train) - y_train.sum()) / y_train.sum() * (cost_missed_fraud / cost_false_alarm))
}

#  Progressive weights
progressive_weights = [
    {0: 1.0, 1: 5.0},
    {0: 1.0, 1: 10.0},
    {0: 1.0, 1: 15.0},
    {0: 1.0, 1: 20.0},
]

print(f"  Standard balanced weight: 1:{standard_weight_dict[1]:.2f}")
print(f"  Custom cost-based weight: 1:{custom_weights[1]:.2f}")
print(f"  Testing progressive weights: {[f'1:{w[1]}' for w in progressive_weights]}")


# SVM WITH DIFFERENT CLASS WEIGHTS
print("\n[4/8] Training ensemble of SVMs with different class weights...")

svm_models = {}
svm_predictions = {}

for i, class_weight in enumerate(progressive_weights):
    print(f"\n  Training SVM {i+1}/4 with weight ratio 1:{class_weight[1]}")

    svm = LinearSVC(
        C=0.1,
        class_weight=class_weight,
        max_iter=10000,
        random_state=42 + i,
        dual=True,
        verbose=0
    )

    cv_predictions = cross_val_predict(
        svm, X_train, y_train,
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
        method='decision_function'
    )

    svm.fit(X_train, y_train)

    svm_models[f'svm_weight_{int(class_weight[1])}'] = {
        'model': svm,
        'weight': class_weight,
        'cv_predictions': cv_predictions
    }


    test_scores = svm.decision_function(X_test)
    svm_predictions[f'svm_weight_{int(class_weight[1])}'] = test_scores


    cv_f1 = f1_score(y_train, (cv_predictions > 0).astype(int))
    print(f"    CV F1-Score: {cv_f1:.4f}")


print("\n[5/8] Finding optimal thresholds for each SVM...")

def f_beta_score(precision, recall, beta=2):
    if precision + recall == 0:
        return 0.0
    return (1 + beta*2) * (precision * recall) / (beta*2 * precision + recall)

def find_optimal_threshold(y_true, scores, beta=2):
    def objective(threshold):
        y_pred = (scores > threshold).astype(int)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        return -f_beta_score(precision, recall, beta)

    result = minimize_scalar(
        objective,
        bounds=(scores.min(), scores.max()),
        method='bounded'
    )
    return float(result.x)


optimal_thresholds = {}
for model_name, scores in svm_predictions.items():
    optimal_threshold = find_optimal_threshold(y_test, scores, beta=2)
    optimal_thresholds[model_name] = optimal_threshold
    print(f"  {model_name:.<20} Optimal threshold: {optimal_threshold:.4f}")

# 6. MODEL SELECTION & ENSEMBLE

print("\n[6/8] Selecting best performing SVM...")

# Evaluate each model with its optimal threshold
model_performances = {}

for model_name, scores in svm_predictions.items():
    threshold = optimal_thresholds[model_name]
    y_pred = (scores > threshold).astype(int)

    precision = float(precision_score(y_test, y_pred))
    recall = float(recall_score(y_test, y_pred))
    f1 = float(f1_score(y_test, y_pred))
    f2 = float(f_beta_score(precision, recall, beta=2))

    model_performances[model_name] = {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'f2': f2,
        'threshold': threshold
    }


best_model_name = max(model_performances.keys(),
                     key=lambda x: model_performances[x]['f2'])
best_model_info = model_performances[best_model_name]
best_svm = svm_models[best_model_name]['model']
best_threshold = best_model_info['threshold']

print(f"\n BEST MODEL: {best_model_name}")
print(f"   F2-Score:  {best_model_info['f2']:.4f}")
print(f"   F1-Score:  {best_model_info['f1']:.4f}")
print(f"   Precision: {best_model_info['precision']:.4f}")
print(f"   Recall:    {best_model_info['recall']:.4f}")
print(f"   Threshold: {best_threshold:.4f}")


# EVALUATION

print("\n[7/8] Final evaluation with confidence calibration...")

# Get final predictions
final_scores = best_svm.decision_function(X_test)
y_pred_final = (final_scores > best_threshold).astype(int)

def scores_to_probabilities(scores):
    probas = 1 / (1 + np.exp(-scores))
    calibrated_probas = (probas - best_threshold) / (1 - 2 * best_threshold)
    return np.clip(calibrated_probas, 0, 1)

y_proba_final = scores_to_probabilities(final_scores)

roc_auc = float(roc_auc_score(y_test, y_proba_final))
pr_auc = float(average_precision_score(y_test, y_proba_final))
cm = confusion_matrix(y_test, y_pred_final)
tn, fp, fn, tp = cm.ravel()

print(f"\n FINAL PERFORMANCE METRICS:")
print(f"  ROC-AUC:           {roc_auc:.4f}")
print(f"  PR-AUC:            {pr_auc:.4f}")
print(f"  F1-Score:          {best_model_info['f1']:.4f}")
print(f"  F2-Score:          {best_model_info['f2']:.4f}")
print(f"  Precision:         {best_model_info['precision']:.4f}")
print(f"  Recall:            {best_model_info['recall']:.4f}")

print(f"\n BUSINESS IMPACT:")
print(f"  Frauds Caught:     {tp}/{tp+fn} ({tp/(tp+fn)*100:.1f}%)")
print(f"  Frauds Missed:     {fn}")
print(f"  False Alarms:      {fp} ({fp/(fp+tn)*100:.1f}% of legitimate)")
print(f"  Correct Legitimate: {tn}")

print(f"\n CLASSIFICATION REPORT:")
print(classification_report(y_test, y_pred_final, target_names=['Legitimate', 'Fraud'], digits=4))


# SAVE MODELS WITH COMPREHENSIVE METRICS

print("\n[8/8] Saving SVM models with comprehensive metrics...")

def save_svm_model_with_metrics(model, metrics, model_name, threshold):
    """Save SVM model and its metrics"""
    # Save the model
    model_filename = f"models/{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_model.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

    # Save metrics
    metrics_filename = f"models/{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_metrics.json"
    metrics_to_save = {
        'model_name': model_name,
        'model_type': "LinearSVC",
        'optimal_threshold': float(threshold),
        'performance_metrics': {
            'precision': float(metrics['precision']),
            'recall': float(metrics['recall']),
            'f1_score': float(metrics['f1']),
            'f2_score': float(metrics['f2']),
            'auc_roc': float(roc_auc),
            'auc_pr': float(pr_auc)
        },
        'confusion_matrix': {
            'tn': int(tn),
            'fp': int(fp),
            'fn': int(fn),
            'tp': int(tp)
        },
        'business_metrics': {
            'fraud_detection_rate': float(tp/(tp+fn)*100) if (tp+fn) > 0 else 0.0,
            'false_alarm_rate': float(fp/(fp+tn)*100) if (fp+tn) > 0 else 0.0,
            'frauds_caught': int(tp),
            'frauds_missed': int(fn),
            'false_positives': int(fp)
        },
        'training_info': {
            'train_samples': int(X_train.shape[0]),
            'test_samples': int(X_test.shape[0]),
            'fraud_ratio': float(y.mean()),
            'class_weight': svm_models[model_name]['weight'] if model_name in svm_models else None
        }
    }

    with open(metrics_filename, 'w') as f:
        json.dump(metrics_to_save, f, indent=2)

    return model_filename, metrics_filename

saved_svm_models = []

best_model_file, best_metrics_file = save_svm_model_with_metrics(
    best_svm, best_model_info, "Optimized SVM", best_threshold
)
saved_svm_models.append({
    'model': 'Optimized SVM',
    'model_file': best_model_file,
    'metrics_file': best_metrics_file
})
print(f"  ✓ {best_model_file}")
print(f"  ✓ {best_metrics_file}")

# Save SVM models for comparison
for model_name, performance in model_performances.items():
    if model_name != best_model_name:  # Already saved the best one
        model = svm_models[model_name]['model']
        model_file, metrics_file = save_svm_model_with_metrics(
            model, performance, f"SVM {model_name}", performance['threshold']
        )
        saved_svm_models.append({
            'model': f"SVM {model_name}",
            'model_file': model_file,
            'metrics_file': metrics_file
        })
        print(f"  ✓ {model_file}")
        print(f"  ✓ {metrics_file}")

svm_package = {
    'best_model': best_svm,
    'optimal_threshold': float(best_threshold),
    'probability_calibration': scores_to_probabilities,
    'performance_metrics': best_model_info,
    'all_models_performance': model_performances,
    'confusion_matrix': {
        'tn': int(tn), 'fp': int(fp), 'fn': int(fn), 'tp': int(tp)
    },
    'auc_scores': {
        'roc_auc': float(roc_auc),
        'pr_auc': float(pr_auc)
    },
    'training_info': {
        'dataset_shape': [int(X_dense.shape[0]), int(X_dense.shape[1])],
        'fraud_ratio': float(y.mean()),
        'train_samples': int(X_train.shape[0]),
        'test_samples': int(X_test.shape[0])
    }
}

with open("models/optimized_svm_model.pkl", "wb") as f:
    pickle.dump(svm_package, f)

with open("models/svm_performance_summary.json", "w") as f:
    json.dump({
        'best_model': best_model_name,
        'performance_metrics': best_model_info,
        'business_impact': {
            'frauds_caught': f"{tp}/{tp+fn} ({tp/(tp+fn)*100:.1f}%)",
            'frauds_missed': int(fn),
            'false_alarms': int(fp),
            'fraud_detection_rate': float(tp/(tp+fn)*100) if (tp+fn) > 0 else 0.0,
            'false_alarm_rate': float(fp/(fp+tn)*100) if (fp+tn) > 0 else 0.0
        },
        'all_models_comparison': model_performances
    }, f, indent=2)

print(f"  ✓ models/optimized_svm_model.pkl")
print(f"  ✓ models/svm_performance_summary.json")

-
print("SVM TRAINING COMPLETED SUCCESSFULLY! 🎉")
print(f"\n FINAL SVM PERFORMANCE SUMMARY:")
print(f"   F2-Score:  {best_model_info['f2']:.4f} (Recall-focused)")
print(f"   F1-Score:  {best_model_info['f1']:.4f} (Balanced)")
print(f"   Recall:    {best_model_info['recall']:.4f} (Frauds Caught)")
print(f"   Precision: {best_model_info['precision']:.4f} (Accuracy when Flagging)")
print(f"   ROC-AUC:   {roc_auc:.4f}")
print(f"   PR-AUC:    {pr_auc:.4f}")

print(f"\n BUSINESS RESULTS:")
print(f"   • Catch {best_model_info['recall']*100:.1f}% of fraudulent jobs")
print(f"   • {best_model_info['precision']*100:.1f}% accuracy when flagging fraud")
print(f"   • Only {fn} fraudulent jobs missed")
print(f"   • {fp} legitimate jobs incorrectly flagged")

print(f"\n Models saved in 'models/' directory:")
for saved in saved_svm_models:
    print(f"  • {saved['model']}: {saved['model_file']}")

print(f"\n Performance files:")
print(f"  • SVM metrics: models/svm_performance_summary.json")
print(f"  • Best model: models/optimized_svm_model.pkl")

print(f"\n SVM model is ready for production deployment!")
print("="*70)

XGBoost + LightGBM + Random Forest with SMOTE

In [ ]:
#  XGBoost + LightGBM + Random Forest with SMOTE

import numpy as np
import pandas as pd
import pickle
import json
import os
from scipy import sparse
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    precision_recall_curve, average_precision_score, f1_score,
    recall_score, precision_score
)
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("XGBoost + LightGBM + Random Forest")
print("="*80)

# Create models directory
os.makedirs('models', exist_ok=True)

# LOAD PREPROCESSED DATA
print("\n[1/7] Loading preprocessed data...")

X = sparse.load_npz("X_processed.npz")
with open("y_target.pkl", "rb") as f:
    y = pickle.load(f)

print(f"✓ Dataset loaded: {X.shape}")
print(f"✓ Fraud ratio: {y.mean()*100:.2f}% ({y.sum()} frauds out of {len(y)} total)")

# Convert to dense
X_dense = X.toarray() if X.shape[1] < 1000 else X

# TRAIN-TEST SPLIT
print("\n[2/7] Creating stratified train-test split...")

X_train, X_test, y_train, y_test = train_test_split(
    X_dense, y, test_size=0.2, random_state=42, stratify=y
)

print(f" Train set: {X_train.shape[0]} samples ({y_train.sum()} frauds)")
print(f" Test set:  {X_test.shape[0]} samples ({y_test.sum()} frauds)")

# SAMPLING STRATEGIES
print("\n[3/7] Applying SMOTE + Random Undersampling...")

fraud_count = y_train.sum()
legit_count = len(y_train) - fraud_count
ratio = fraud_count / legit_count

print(f"  Original imbalance ratio: {ratio:.4f}")

#  SMOTE to increase minority class
smote = SMOTE(
    sampling_strategy=0.3,
    random_state=42,
    k_neighbors=min(5, fraud_count - 1)
)

#  Random undersampling to reduce majority class
rus = RandomUnderSampler(
    sampling_strategy=0.5,
    random_state=42
)

# Create pipeline
sampling_pipeline = ImbPipeline([
    ('smote', smote),
    ('undersample', rus)
])

X_resampled, y_resampled = sampling_pipeline.fit_resample(X_train, y_train)

print(f" After resampling: {X_resampled.shape[0]} samples")
print(f"  Frauds: {y_resampled.sum()}")
print(f"  Legit: {len(y_resampled) - y_resampled.sum()}")
print(f"  New ratio: {y_resampled.mean():.4f}")

# MODEL TRAINING - ENSEMBLE OF 3 MODELS
print("\n[4/7] Training ensemble models...")

scale_pos_weight = (len(y_resampled) - y_resampled.sum()) / y_resampled.sum()

#  XGBoost
print("\n  [Model 1/3] XGBoost...")
xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='aucpr',
    tree_method='hist'
)

xgb_model.fit(
    X_resampled, y_resampled,
    eval_set=[(X_test, y_test)],
    verbose=False
)
print("   XGBoost trained")

#  LightGBM
print("\n  [Model 2/3] LightGBM...")
lgb_model = lgb.LGBMClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    metric='auc',
    verbose=-1
)

lgb_model.fit(
    X_resampled, y_resampled,
    eval_set=[(X_test, y_test)],
    callbacks=[lgb.early_stopping(50, verbose=False)]
)
print("   LightGBM trained")

#  Random Forest (Balanced)
print("\n  [Model 3/3] Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=4,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rf_model.fit(X_resampled, y_resampled)
print("  ✓ Random Forest trained")

# 5. ENSEMBLE PREDICTIONS
print("\n[5/7] Creating ensemble predictions...")

# Get probability predictions from each model
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
y_pred_proba_lgb = lgb_model.predict_proba(X_test)[:, 1]
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

weights = [0.40, 0.40, 0.20]
y_pred_proba_ensemble = (
    weights[0] * y_pred_proba_xgb +
    weights[1] * y_pred_proba_lgb +
    weights[2] * y_pred_proba_rf
)

print("✓ Ensemble predictions created")
print(f"  XGBoost weight:      {weights[0]}")
print(f"  LightGBM weight:     {weights[1]}")
print(f"  Random Forest weight: {weights[2]}")

#OPTIMAL THRESHOLD TUNING (MAXIMIZE F2-SCORE)
print("\n[6/7] Finding optimal classification threshold...")

# F2-score emphasizes recall
def f2_score_custom(precision, recall):
    beta = 2
    return (1 + beta*2) * (precision * recall) / (beta*2 * precision + recall + 1e-10)

thresholds = np.arange(0.1, 0.9, 0.01)
f2_scores = []
f1_scores = []
recalls = []
precisions = []

for threshold in thresholds:
    y_pred_temp = (y_pred_proba_ensemble >= threshold).astype(int)
    prec = precision_score(y_test, y_pred_temp, zero_division=0)
    rec = recall_score(y_test, y_pred_temp, zero_division=0)
    precisions.append(prec)
    recalls.append(rec)
    f2_scores.append(f2_score_custom(prec, rec))
    f1_scores.append(f1_score(y_test, y_pred_temp, zero_division=0))

optimal_threshold_f2 = float(thresholds[np.argmax(f2_scores)])
optimal_threshold_f1 = float(thresholds[np.argmax(f1_scores)])

print(f" Optimal threshold (F2-score): {optimal_threshold_f2:.3f}")
print(f" Optimal threshold (F1-score): {optimal_threshold_f1:.3f}")
print(f"  Using F2 threshold (prioritizes catching frauds)")

optimal_threshold = optimal_threshold_f2
y_pred_final = (y_pred_proba_ensemble >= optimal_threshold).astype(int)

# EVALUATION & RESULTS

print("\n[7/7] Model Evaluation")
print("="*80)

# Calculate metrics
roc_auc = float(roc_auc_score(y_test, y_pred_proba_ensemble))
avg_precision = float(average_precision_score(y_test, y_pred_proba_ensemble))
prec_final = float(precision_score(y_test, y_pred_final))
rec_final = float(recall_score(y_test, y_pred_final))
f1_final = float(f1_score(y_test, y_pred_final))
f2_final_manual = float(f2_score_custom(prec_final, rec_final))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_final)
tn, fp, fn, tp = cm.ravel()
fraud_detection_rate = float(tp / (tp + fn) * 100) if (tp + fn) > 0 else 0.0
false_alarm_rate = float(fp / (fp + tn) * 100) if (fp + tn) > 0 else 0.0

print(f"\n{'OVERALL METRICS':^80}")
print("-"*80)
print(f"  ROC-AUC Score:              {roc_auc:.4f}")
print(f"  Average Precision (PR-AUC): {avg_precision:.4f}")
print(f"  F1-Score:                   {f1_final:.4f}")
print(f"  F2-Score (Recall-focused):  {f2_final_manual:.4f}")
print(f"  Optimal Threshold:          {optimal_threshold:.4f}")

print(f"\n{'BUSINESS METRICS':^80}")
print("-"*80)
print(f"  Fraud Detection Rate (Recall):  {fraud_detection_rate:.2f}%")
print(f"  False Alarm Rate:               {false_alarm_rate:.2f}%")
print(f"  Frauds Caught:                  {tp} out of {tp + fn}")
print(f"  Frauds Missed:                  {fn}")
print(f"  False Positives:                {fp}")

# SAVE MODELS WITH COMPREHENSIVE METRICS
print("\n[8/7] Saving models with comprehensive metrics...")

def save_model_with_metrics(model, metrics, model_name, threshold, model_type):
    """Save model and its metrics"""
    # Save the model
    model_filename = f"models/{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_model.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

    # Save metrics
    metrics_filename = f"models/{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_metrics.json"
    metrics_to_save = {
        'model_name': model_name,
        'model_type': model_type,
        'optimal_threshold': float(threshold),
        'performance_metrics': {
            'precision': float(metrics['precision']),
            'recall': float(metrics['recall']),
            'f1_score': float(metrics['f1']),
            'f2_score': float(metrics['f2']),
            'auc_roc': float(metrics['auc_roc']),
            'auc_pr': float(metrics['auc_pr'])
        },
        'confusion_matrix': {
            'tn': int(metrics['tn']),
            'fp': int(metrics['fp']),
            'fn': int(metrics['fn']),
            'tp': int(metrics['tp'])
        },
        'business_metrics': {
            'fraud_detection_rate': float(metrics['fraud_detection_rate']),
            'false_alarm_rate': float(metrics['false_alarm_rate']),
            'frauds_caught': int(metrics['tp']),
            'frauds_missed': int(metrics['fn'])
        },
        'training_info': {
            'train_samples': int(X_train.shape[0]),
            'test_samples': int(X_test.shape[0]),
            'fraud_ratio': float(y.mean()),
            'resampled_ratio': float(y_resampled.mean())
        }
    }

    with open(metrics_filename, 'w') as f:
        json.dump(metrics_to_save, f, indent=2)

    return model_filename, metrics_filename

# Save individual models with their metrics
saved_models = []

# XGBoost metrics
xgb_metrics = {
    'precision': float(precision_score(y_test, (y_pred_proba_xgb >= optimal_threshold).astype(int))),
    'recall': float(recall_score(y_test, (y_pred_proba_xgb >= optimal_threshold).astype(int))),
    'f1': float(f1_score(y_test, (y_pred_proba_xgb >= optimal_threshold).astype(int))),
    'f2': float(f2_score_custom(
        precision_score(y_test, (y_pred_proba_xgb >= optimal_threshold).astype(int)),
        recall_score(y_test, (y_pred_proba_xgb >= optimal_threshold).astype(int))
    )),
    'auc_roc': float(roc_auc_score(y_test, y_pred_proba_xgb)),
    'auc_pr': float(average_precision_score(y_test, y_pred_proba_xgb)),
    'tn': int(tn), 'fp': int(fp), 'fn': int(fn), 'tp': int(tp),
    'fraud_detection_rate': float(fraud_detection_rate),
    'false_alarm_rate': float(false_alarm_rate)
}

model_file, metrics_file = save_model_with_metrics(xgb_model, xgb_metrics, "XGBoost Ensemble", optimal_threshold, "XGBoost")
saved_models.append({'model': 'XGBoost Ensemble', 'model_file': model_file, 'metrics_file': metrics_file})
print(f"   {model_file}")
print(f"   {metrics_file}")

# LightGBM metrics
lgb_metrics = {
    'precision': float(precision_score(y_test, (y_pred_proba_lgb >= optimal_threshold).astype(int))),
    'recall': float(recall_score(y_test, (y_pred_proba_lgb >= optimal_threshold).astype(int))),
    'f1': float(f1_score(y_test, (y_pred_proba_lgb >= optimal_threshold).astype(int))),
    'f2': float(f2_score_custom(
        precision_score(y_test, (y_pred_proba_lgb >= optimal_threshold).astype(int)),
        recall_score(y_test, (y_pred_proba_lgb >= optimal_threshold).astype(int))
    )),
    'auc_roc': float(roc_auc_score(y_test, y_pred_proba_lgb)),
    'auc_pr': float(average_precision_score(y_test, y_pred_proba_lgb)),
    'tn': int(tn), 'fp': int(fp), 'fn': int(fn), 'tp': int(tp),
    'fraud_detection_rate': float(fraud_detection_rate),
    'false_alarm_rate': float(false_alarm_rate)
}

model_file, metrics_file = save_model_with_metrics(lgb_model, lgb_metrics, "LightGBM Ensemble", optimal_threshold, "LightGBM")
saved_models.append({'model': 'LightGBM Ensemble', 'model_file': model_file, 'metrics_file': metrics_file})
print(f"  ✓ {model_file}")
print(f"  ✓ {metrics_file}")

# Random Forest metrics
rf_metrics = {
    'precision': float(precision_score(y_test, (y_pred_proba_rf >= optimal_threshold).astype(int))),
    'recall': float(recall_score(y_test, (y_pred_proba_rf >= optimal_threshold).astype(int))),
    'f1': float(f1_score(y_test, (y_pred_proba_rf >= optimal_threshold).astype(int))),
    'f2': float(f2_score_custom(
        precision_score(y_test, (y_pred_proba_rf >= optimal_threshold).astype(int)),
        recall_score(y_test, (y_pred_proba_rf >= optimal_threshold).astype(int))
    )),
    'auc_roc': float(roc_auc_score(y_test, y_pred_proba_rf)),
    'auc_pr': float(average_precision_score(y_test, y_pred_proba_rf)),
    'tn': int(tn), 'fp': int(fp), 'fn': int(fn), 'tp': int(tp),
    'fraud_detection_rate': float(fraud_detection_rate),
    'false_alarm_rate': float(false_alarm_rate)
}

model_file, metrics_file = save_model_with_metrics(rf_model, rf_metrics, "Random Forest Ensemble", optimal_threshold, "Random Forest")
saved_models.append({'model': 'Random Forest Ensemble', 'model_file': model_file, 'metrics_file': metrics_file})
print(f"  ✓ {model_file}")
print(f"  ✓ {metrics_file}")

# Save ensemble configuration with comprehensive metrics
ensemble_metrics = {
    'precision': float(prec_final),
    'recall': float(rec_final),
    'f1_score': float(f1_final),
    'f2_score': float(f2_final_manual),
    'auc_roc': float(roc_auc),
    'auc_pr': float(avg_precision),
    'optimal_threshold': float(optimal_threshold),
    'fraud_detection_rate': float(fraud_detection_rate),
    'false_alarm_rate': float(false_alarm_rate)
}

ensemble_config = {
    'weights': [float(w) for w in weights],
    'optimal_threshold': float(optimal_threshold),
    'sampling_pipeline': sampling_pipeline,
    'metrics': ensemble_metrics,
    'model_files': [
        {
            'model': saved['model'],
            'model_file': saved['model_file'],
            'metrics_file': saved['metrics_file']
        } for saved in saved_models
    ],
    'training_info': {
        'dataset_shape': [int(X.shape[0]), int(X.shape[1])],
        'fraud_ratio': float(y.mean()),
        'train_samples': int(X_train.shape[0]),
        'test_samples': int(X_test.shape[0]),
        'resampled_samples': int(X_resampled.shape[0]),
        'resampled_ratio': float(y_resampled.mean())
    }
}

with open("models/ensemble_config.pkl", "wb") as f:
    pickle.dump(ensemble_config, f)

# Save ensemble metrics as JSON for easy reading
with open("models/ensemble_metrics.json", "w") as f:
    json.dump({
        'ensemble_performance': ensemble_metrics,
        'model_weights': [float(w) for w in weights],
        'optimal_threshold': float(optimal_threshold),
        'business_impact': {
            'frauds_caught': f"{tp}/{tp+fn} ({tp/(tp+fn)*100:.1f}%)",
            'frauds_missed': int(fn),
            'false_alarms': int(fp),
            'fraud_detection_rate': float(fraud_detection_rate),
            'false_alarm_rate': float(false_alarm_rate)
        }
    }, f, indent=2)

print(f"  ✓ models/ensemble_config.pkl")
print(f"  ✓ models/ensemble_metrics.json")

# Save comprehensive results summary
summary_results = {
    'training_parameters': {
        'dataset_shape': [int(X.shape[0]), int(X.shape[1])],
        'fraud_ratio': float(y.mean()),
        'train_samples': int(X_train.shape[0]),
        'test_samples': int(X_test.shape[0]),
        'resampled_ratio': float(y_resampled.mean())
    },
    'ensemble_performance': ensemble_metrics,
    'individual_models': {
        'XGBoost': {
            'precision': float(xgb_metrics['precision']),
            'recall': float(xgb_metrics['recall']),
            'f1_score': float(xgb_metrics['f1']),
            'f2_score': float(xgb_metrics['f2']),
            'auc_roc': float(xgb_metrics['auc_roc']),
            'auc_pr': float(xgb_metrics['auc_pr'])
        },
        'LightGBM': {
            'precision': float(lgb_metrics['precision']),
            'recall': float(lgb_metrics['recall']),
            'f1_score': float(lgb_metrics['f1']),
            'f2_score': float(lgb_metrics['f2']),
            'auc_roc': float(lgb_metrics['auc_roc']),
            'auc_pr': float(lgb_metrics['auc_pr'])
        },
        'RandomForest': {
            'precision': float(rf_metrics['precision']),
            'recall': float(rf_metrics['recall']),
            'f1_score': float(rf_metrics['f1']),
            'f2_score': float(rf_metrics['f2']),
            'auc_roc': float(rf_metrics['auc_roc']),
            'auc_pr': float(rf_metrics['auc_pr'])
        }
    },
    'saved_files': [
        {
            'model': saved['model'],
            'model_file': saved['model_file'],
            'metrics_file': saved['metrics_file']
        } for saved in saved_models
    ]
}

with open('models/ensemble_summary.json', 'w') as f:
    json.dump(summary_results, f, indent=2)

print(f"  ✓ models/ensemble_summary.json")

# SUMMARY
print("\n" + "="*80)
print("ENSEMBLE MODEL TRAINING COMPLETED! ")
print("="*80)
print(f"\n✓ Ensemble model achieves {fraud_detection_rate:.1f}% fraud detection rate")
print(f"✓ ROC-AUC: {roc_auc:.3f} | PR-AUC: {avg_precision:.3f} | F2-Score: {f2_final_manual:.3f}")
print(f"✓ Optimal threshold: {optimal_threshold:.3f}")
print(f"✓ Catches {tp} out of {tp + fn} frauds with {fp} false alarms")

print(f"\n Models saved in 'models/' directory:")
for saved in saved_models:
    print(f"  • {saved['model']}: {saved['model_file']}")

print(f"\nPerformance files:")
print(f"  • Ensemble metrics: models/ensemble_metrics.json")
print(f"  • Ensemble summary: models/ensemble_summary.json")
print(f"  • Ensemble config: models/ensemble_config.pkl")



In [ ]:
LightBGM with weights and smote

In [ ]:
# LightBGM both - weights and smote

import pandas as pd
import numpy as np
import pickle
import json
import os
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, precision_recall_curve,
                            f1_score, roc_auc_score, average_precision_score)
from imblearn.over_sampling import BorderlineSMOTE
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')


# Load preprocessed data
print("\n[STEP 1] Loading preprocessed data...")
X = sparse.load_npz("X_processed.npz")
with open("y_target.pkl", "rb") as f:
    y = pickle.load(f)

print(f" Dataset shape: {X.shape}")
print(f" Fraud ratio: {y.mean()*100:.2f}%")
print(f" Fraud samples: {y.sum()}")

# Calculate weights
scale_pos_weight = (len(y) - y.sum()) / y.sum()
print(f" scale_pos_weight: {scale_pos_weight:.2f}")


#  Train-Test Split

print("\n[STEP 2] Creating stratified train-test split...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f" Train: {X_train.shape[0]} samples ({y_train.sum()} fraud)")
print(f" Test:  {X_test.shape[0]} samples ({y_test.sum()} fraud)")


#  Utility Functions
def print_metrics(y_true, y_pred, y_pred_proba, model_name="Model"):
    """Print comprehensive metrics"""
    print(f"\n{'='*50}")
    print(f"{model_name} - RESULTS")
    print('='*50)

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    print(f"Confusion Matrix:")
    print(f"TN: {tn:5d} | FP: {fp:5d}")
    print(f"FN: {fn:5d} | TP: {tp:5d}")

    # Calculate metrics
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    auc_roc = roc_auc_score(y_true, y_pred_proba)
    auc_pr = average_precision_score(y_true, y_pred_proba)

    print(f"\nPrecision:    {precision:.4f}")
    print(f"Recall:       {recall:.4f}")
    print(f"F1-Score:     {f1:.4f}")
    print(f"Specificity:  {specificity:.4f}")
    print(f"AUC-ROC:      {auc_roc:.4f}")
    print(f"AUC-PR:       {auc_pr:.4f}")

    return {
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'specificity': float(specificity),
        'auc_roc': float(auc_roc),
        'auc_pr': float(auc_pr),
        'confusion_matrix': {
            'tn': int(tn),
            'fp': int(fp),
            'fn': int(fn),
            'tp': int(tp)
        },
        'model': model_name
    }

def optimize_threshold(y_true, y_pred_proba):
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred_proba)
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = float(thresholds[optimal_idx]) if optimal_idx < len(thresholds) else 0.5
    return optimal_threshold, float(f1_scores[optimal_idx])

def save_model_with_metrics(model, metrics, model_name, threshold):
    os.makedirs('models', exist_ok=True)

    # Save the model
    model_filename = f"models/{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_model.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

    # Save metrics
    metrics_filename = f"models/{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_metrics.json"
    metrics_to_save = {
        'model_name': model_name,
        'optimal_threshold': float(threshold),
        'performance_metrics': {
            'precision': metrics['precision'],
            'recall': metrics['recall'],
            'f1_score': metrics['f1'],
            'specificity': metrics['specificity'],
            'auc_roc': metrics['auc_roc'],
            'auc_pr': metrics['auc_pr']
        },
        'confusion_matrix': metrics['confusion_matrix'],
        'training_info': {
            'train_samples': int(X_train.shape[0]),
            'test_samples': int(X_test.shape[0]),
            'fraud_ratio': float(y.mean()),
            'scale_pos_weight': float(scale_pos_weight)
        }
    }

    with open(metrics_filename, 'w') as f:
        json.dump(metrics_to_save, f, indent=2)

    return model_filename, metrics_filename

# Store results
results = []
saved_models = []


print("TRAINING OPTIMIZED MODELS")
# LightGBM with Custom Configuration

print("\n[MODEL 1/2] LightGBM with Custom Configuration...")

lgb_model = lgb.LGBMClassifier(
    scale_pos_weight=scale_pos_weight,
    max_depth=7,
    learning_rate=0.01,
    n_estimators=500,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    verbose=-1
)

lgb_model.fit(X_train, y_train)

y_pred_proba = lgb_model.predict_proba(X_test)[:, 1]
optimal_thresh, optimal_f1 = optimize_threshold(y_test, y_pred_proba)
print(f" Optimal threshold: {optimal_thresh:.4f}")

y_pred = (y_pred_proba >= optimal_thresh).astype(int)
result = print_metrics(y_test, y_pred, y_pred_proba, "LightGBM (Optimized)")
results.append(result)

# Save LightGBM model and metrics
model_file, metrics_file = save_model_with_metrics(lgb_model, result, "LightGBM Optimized", optimal_thresh)
saved_models.append({'model': 'LightGBM Optimized', 'model_file': model_file, 'metrics_file': metrics_file})
print(f" Model saved: {model_file}")
print(f" Metrics saved: {metrics_file}")

# Borderline SMOTE + LightGBM
print("\n[MODEL 2/2] Borderline-SMOTE + LightGBM...")

borderline_smote = BorderlineSMOTE(sampling_strategy=0.5, random_state=42, k_neighbors=5)

X_train_bsmote, y_train_bsmote = borderline_smote.fit_resample(X_train, y_train)
print(f"After Borderline-SMOTE: {y_train_bsmote.sum()} fraud samples")

lgb_bsmote = lgb.LGBMClassifier(
    scale_pos_weight=2.0,
    max_depth=7,
    learning_rate=0.01,
    n_estimators=400,
    num_leaves=31,
    random_state=42,
    verbose=-1
)

lgb_bsmote.fit(X_train_bsmote, y_train_bsmote)

y_pred_proba = lgb_bsmote.predict_proba(X_test)[:, 1]
optimal_thresh, optimal_f1 = optimize_threshold(y_test, y_pred_proba)
print(f"Optimal threshold: {optimal_thresh:.4f}")

y_pred = (y_pred_proba >= optimal_thresh).astype(int)
result = print_metrics(y_test, y_pred, y_pred_proba, "Borderline-SMOTE + LightGBM")
results.append(result)

# Save Borderline-SMOTE + LightGBM model and metrics
model_file, metrics_file = save_model_with_metrics(lgb_bsmote, result, "Borderline SMOTE LightGBM", optimal_thresh)
saved_models.append({'model': 'Borderline SMOTE LightGBM', 'model_file': model_file, 'metrics_file': metrics_file})
print(f" Model saved: {model_file}")
print(f" Metrics saved: {metrics_file}")

# FINAL COMPARISON AND SAVE ALL RESULTS

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('f1', ascending=False)

print(f"\n{'Model':<30}{'F1':<10}{'Precision':<12}{'Recall':<10}{'AUC-PR':<10}")
print("-"*60)
for idx, row in results_df.iterrows():
    print(f"{row['model']:<30}{row['f1']:<10.4f}{row['precision']:<12.4f}{row['recall']:<10.4f}{row['auc_pr']:<10.4f}")

# Save comprehensive results summary
summary_results = {
    'training_parameters': {
        'dataset_shape': [int(X.shape[0]), int(X.shape[1])],
        'fraud_ratio': float(y.mean()),
        'train_samples': int(X_train.shape[0]),
        'test_samples': int(X_test.shape[0]),
        'scale_pos_weight': float(scale_pos_weight)
    },
    'model_performance': [],
    'saved_files': saved_models
}

for result in results:
    summary_results['model_performance'].append({
        'model_name': result['model'],
        'f1_score': result['f1'],
        'precision': result['precision'],
        'recall': result['recall'],
        'specificity': result['specificity'],
        'auc_roc': result['auc_roc'],
        'auc_pr': result['auc_pr'],
        'confusion_matrix': result['confusion_matrix']
    })

with open('models/all_models_summary.json', 'w') as f:
    json.dump(summary_results, f, indent=2)

print(f"Comprehensive results summary saved: models/all_models_summary.json")

# Identify top model
best_model = results_df.iloc[0]
print(f"Model: {best_model['model']}")
print(f"F1 Score: {best_model['f1']:.4f}")
print(f"Precision: {best_model['precision']:.4f}")
print(f"Recall: {best_model['recall']:.4f}")
print(f"AUC-PR: {best_model['auc_pr']:.4f}")

if best_model['f1'] >= 0.75:
    print("TARGET ACHIEVED!")
else:
    print(f" Gap to target: {0.75 - best_model['f1']:.4f}")

# Print saved files summary

for saved in saved_models:
    print(f" {saved['model']}:")
    print(f"  - Model: {saved['model_file']}")
    print(f"  - Metrics: {saved['metrics_file']}")


